<a href="https://colab.research.google.com/github/DeisyData/BIT_IA_Bootcamp/blob/main/Plantilla_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [1]:

from transformers import AutoModelForCausalLM, AutoTokenizer, BlenderbotForConditionalGeneration
import torch
import gradio as gr


# Modelo y Tokenizer
chat_tkn = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
mdl = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Función de conversación
def converse(user_input, chat_history=None):
    if chat_history is None:
        chat_history = []

    user_input_ids = chat_tkn(user_input + chat_tkn.eos_token, return_tensors='pt').input_ids
    if len(chat_history) > 0:
        bot_input_ids = torch.cat([torch.LongTensor(chat_history), user_input_ids], dim=-1)
    else:
        bot_input_ids = user_input_ids

    chat_history_tensor = mdl.generate(bot_input_ids, max_length=1000, pad_token_id=chat_tkn.eos_token_id)
    chat_history = chat_history_tensor.tolist()

    response = chat_tkn.decode(chat_history[0]).split("<|endoftext|>")
    html = "<div class='mychat'>"
    for x, mesg in enumerate(response):
        if x % 2 != 0:
            mesg = "Alicia: " + mesg
            clazz = "alicia"
        else:
            clazz = "user"
        html += f"<div class='mesg {clazz}'> {mesg}</div>"
    html += "</div>"
    return html, chat_history

# Estilo CSS
css = """
.mychat {display:flex;flex-direction:column}
.mesg {padding:5px;margin-bottom:5px;border-radius:5px;width:75%}
.mesg.user {background-color:lightblue;color:white}
.mesg.alicia {background-color:orange;color:white;align-self:flex-end}
.footer {display:none !important}
"""

# Interfaz Gradio
interface = gr.Interface(
    fn=converse,
    inputs=[gr.Textbox(label="User Input", placeholder="Let's chat"), gr.State()],
    outputs=[gr.HTML(label="Response"), gr.State()],
    css=css,
)

# Lanzar la interfaz
interface.launch()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1d1298e42324787c0c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
